In [1]:
!pip install flask transformers requests

In [2]:
%%writefile flask_chatbot_app.py
from flask import Flask, request, jsonify
from transformers import pipeline, TFAutoModelForSeq2SeqLM, AutoTokenizer

app = Flask(__name__)

# Load the Hugging Face model

model_name = 'google/flan-t5-xl'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)
sentiment_model = pipeline("sentiment-analysis")

@app.route('/chatbot', methods=['POST'])
def chatbot():
    user_input = request.json.get('message')
    if not user_input:
        return jsonify({'response': 'Please provide a message.'}), 400
    sentiment = sentiment_model(user_input)[0]
    response = generate_response(user_input, sentiment)
    return jsonify({'response': response})

def generate_response(user_input, sentiment):
    if sentiment['label'] == 'NEGATIVE':
        prompt = f"The user is upset, respond with empathy and support: {user_input}"
    else:
        prompt = f"Respond to the following query: {user_input}"
    input_ids = tokenizer.encode(prompt, truncation=True, padding=True, max_length=512, return_tensors="tf")
    output = model.generate(input_ids, max_length=250, num_beams=5, early_stopping=True)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response


if __name__ == '__main__':
    app.run(debug=True, port=5000, host='0.0.0.0')


Writing flask_chatbot_app.py


In [3]:

import subprocess

# Stop any running Flask app
subprocess.run(['pkill', '-f', 'flask_chatbot_app.py'])

CompletedProcess(args=['pkill', '-f', 'flask_chatbot_app.py'], returncode=1)

In [4]:
!nohup python flask_chatbot_app.py &

nohup: appending output to 'nohup.out'


In [7]:
!sudo lsof -i -P -n | grep LISTEN

node         7 root   21u  IPv6  14068      0t0  TCP *:8080 (LISTEN)
kernel_ma   14 root    6u  IPv4    699      0t0  TCP 172.28.0.12:6000 (LISTEN)
colab-fil   80 root    3u  IPv4  14070      0t0  TCP 127.0.0.1:3453 (LISTEN)
jupyter-n  136 root    7u  IPv4  22309      0t0  TCP 172.28.0.12:9000 (LISTEN)
python3   3425 root   21u  IPv4 108147      0t0  TCP 127.0.0.1:33215 (LISTEN)
python3   3468 root    3u  IPv4 123047      0t0  TCP 127.0.0.1:35463 (LISTEN)
python3   3468 root    5u  IPv4 123048      0t0  TCP 127.0.0.1:46185 (LISTEN)
pt_main_t 4237 root   33u  IPv4 157786      0t0  TCP *:5000 (LISTEN)
pt_main_t 4743 root   33u  IPv4 157786      0t0  TCP *:5000 (LISTEN)


In [8]:

import requests

# Define the URL of the Flask app
url = 'http://127.0.0.1:5000/chatbot'

# Send a request to the Flask app
response = requests.post(url, json={'message': 'Hello, how are you?'})
print(response.json())


{'response': 'Hello, how are you?'}


In [9]:
response = requests.post(url, json={'message': 'I was wondering how can I go to Eiffel Tower from the airport using the train and subway?'})
print(response.json())

{'response': 'The best way to get to the Eiffel Tower from the airport is by train. You can take the RER B train to the Eiffel Tower and then change to the RER C train to the Eiffel Tower.'}
